In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

2025-06-28 10:58:57.052916: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751108337.316901      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751108337.389970      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
train_path = '/kaggle/input/vcor-vehicle-color-recognition-dataset/test'
val_path = '/kaggle/input/vcor-vehicle-color-recognition-dataset/val'

In [3]:
img_size = (128, 128)
batch_size = 32
datagen = ImageDataGenerator(rescale=1./255)
train_gen = datagen.flow_from_directory(train_path, target_size=img_size, batch_size=batch_size, class_mode='categorical')
val_gen = datagen.flow_from_directory(val_path, target_size=img_size, batch_size=batch_size, class_mode='categorical')

Found 1556 images belonging to 15 classes.
Found 1550 images belonging to 15 classes.


In [5]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(train_gen.num_classes, activation='softmax')
])

In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_gen, validation_data=val_gen, epochs=10)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 45s 866ms/step - accuracy: 0.2320 - loss: 2.3072 - val_accuracy: 0.5181 - val_loss: 1.3035
Epoch 2/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 38s 785ms/step - accuracy: 0.5952 - loss: 1.1389 - val_accuracy: 0.6258 - val_loss: 1.0971
Epoch 3/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 39s 792ms/step - accuracy: 0.6925 - loss: 0.8848 - val_accuracy: 0.6310 - val_loss: 1.0133
Epoch 4/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 37s 758ms/step - accuracy: 0.7564 - loss: 0.6705 - val_accuracy: 0.6723 - val_loss: 0.9853
Epoch 5/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 38s 769ms/step - accuracy: 0.8334 - loss: 0.4896 - val_accuracy: 0.6819 - val_loss: 0.9524
Epoch 6/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 40s 748ms/step - accuracy: 0.8916 - loss: 0.3390 - val_accuracy: 0.6794 - val_loss: 1.0867
Epoch 7/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 36s 746ms/step - accuracy: 0.9291 - loss: 0.2317 - val_accuracy: 0.6619 - val_loss: 1.2015
Epoch 8/10
49/49 ━━━━━━━━━━━━━━━━━━━━ 37s 749ms/step - accuracy: 0.9585 - loss: 0.1433 - val_accu

In [7]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
val_steps = val_gen.samples // val_gen.batch_size
pred_probs = model.predict(val_gen, steps=val_steps+1)
y_pred = np.argmax(pred_probs, axis=1)
y_true = val_gen.classes[:len(y_pred)]

49/49 ━━━━━━━━━━━━━━━━━━━━ 10s 195ms/step


In [9]:
class_names = list(val_gen.class_indices.keys())
loss, acc = model.evaluate(val_gen)
print(f"\nValidation Accuracy: {acc * 100:.2f}%")

49/49 ━━━━━━━━━━━━━━━━━━━━ 10s 192ms/step - accuracy: 0.6833 - loss: 1.3709

Validation Accuracy: 69.68%


In [15]:
model.save("car_color_model.h5")